This notebook demonstrates how to use the **``openmc.mgxs``** module to generate multi-group cross sections with OpenMC.

**Note:** that this Notebook was created using [OpenMOC](https://mit-crpg.github.io/OpenMOC/) to verify the multi-group cross-sections generated by OpenMC. In order to run this Notebook, you must have [OpenMOC](https://mit-crpg.github.io/OpenMOC/) installed on your system, along with OpenCG to convert the OpenMC geometries into OpenMOC geometries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import openmc
import openmc.mgxs as mgxs

%matplotlib inline

# Infinite Homogeneous Medium

We first construct a simple homogeneous infinite medium problem to illustrate use of the `openmc.mgxs` module to generate multi-group cross sections.

### Generate Inputs

First we need to define materials that will be used in the problem. Before defining a material, we must create nuclides that are used in the material.

In [2]:
# Instantiate some Nuclides
h1 = openmc.Nuclide('H-1')
o16 = openmc.Nuclide('O-16')
u235 = openmc.Nuclide('U-235')
u238 = openmc.Nuclide('U-238')
zr90 = openmc.Nuclide('Zr-90')

With the nuclides we defined, we will now create a material for the homogeneous medium.

In [3]:
# Instantiate a Material and register the Nuclides
inf_medium = openmc.Material(name='moderator')
inf_medium.set_density('g/cc', 5.)
inf_medium.add_nuclide(h1,  0.028999667)
inf_medium.add_nuclide(o16, 0.01450188)
inf_medium.add_nuclide(u235, 0.000114142)
inf_medium.add_nuclide(u238, 0.006886019)
inf_medium.add_nuclide(zr90, 0.002116053)

With our material, we can now create a materials file object that can be exported to an actual XML file.

In [4]:
# Instantiate a MaterialsFile, register all Materials, and export to XML
materials_file = openmc.MaterialsFile()
materials_file.default_xs = '71c'
materials_file.add_material(inf_medium)
materials_file.export_to_xml()

Now let's move on to the geometry. This problem will be a simple square cell with reflective boundary conditions to simulate an infinite homogeneous medium. The first step is to create the outer bounding surfaces of the problem.

In [5]:
# Instantiate boundary Planes
min_x = openmc.XPlane(boundary_type='reflective', x0=-0.63)
max_x = openmc.XPlane(boundary_type='reflective', x0=0.63)
min_y = openmc.YPlane(boundary_type='reflective', y0=-0.63)
max_y = openmc.YPlane(boundary_type='reflective', y0=0.63)

With the surfaces defined, we can now create a cell that is defined by intersections of half-spaces created by the surfaces.

In [6]:
# Instantiate a Cell
cell = openmc.Cell(cell_id=1, name='cell')

# Register bounding Surfaces with the Cell
cell.add_surface(surface=min_x, halfspace=+1)
cell.add_surface(surface=max_x, halfspace=-1)
cell.add_surface(surface=min_y, halfspace=+1)
cell.add_surface(surface=max_y, halfspace=-1)

# Fill the Cell with the Material
cell.fill = inf_medium

/usr/local/lib/python2.7/dist-packages/openmc-0.7.0-py2.7.egg/openmc/universe.py:223: DeprecationWarning: Cell.add_surface(...) has been deprecated and may be removed in a future version. The region for a Cell should be defined using the region property directly.
/usr/local/lib/python2.7/dist-packages/openmc-0.7.0-py2.7.egg/openmc/universe.py:223: DeprecationWarning: Cell.add_surface(...) has been deprecated and may be removed in a future version. The region for a Cell should be defined using the region property directly.
/usr/local/lib/python2.7/dist-packages/openmc-0.7.0-py2.7.egg/openmc/universe.py:223: DeprecationWarning: Cell.add_surface(...) has been deprecated and may be removed in a future version. The region for a Cell should be defined using the region property directly.
/usr/local/lib/python2.7/dist-packages/openmc-0.7.0-py2.7.egg/openmc/universe.py:223: DeprecationWarning: Cell.add_surface(...) has been deprecated and may be removed in a future version. The region for a Cel

OpenMC requires that there is a "root" universe. Let us create a root universe and add our square cell to it.

In [7]:
# Instantiate Universe
root_universe = openmc.Universe(universe_id=0, name='root universe')
root_universe.add_cell(cell)

We now must create a geometry that is assigned a root universe, put the geometry into a geometry file, and export it to XML.

In [8]:
# Create Geometry and set root Universe
openmc_geometry = openmc.Geometry()
openmc_geometry.root_universe = root_universe

# Instantiate a GeometryFile
geometry_file = openmc.GeometryFile()
geometry_file.geometry = openmc_geometry

# Export to "geometry.xml"
geometry_file.export_to_xml()

Next, we must define simulation parameters. In this case, we will use 10 inactive batches and 40 active batches each with 2500 particles.

In [9]:
# OpenMC simulation parameters
batches = 50
inactive = 10
particles = 2500

# Instantiate a SettingsFile
settings_file = openmc.SettingsFile()
settings_file.batches = batches
settings_file.inactive = inactive
settings_file.particles = particles
settings_file.output = {'tallies': True, 'summary': True}
bounds = [-0.63, -0.63, -0.63, 0.63, 0.63, 0.63]
settings_file.set_source_space('box', bounds)

# Export to "settings.xml"
settings_file.export_to_xml()

Now we are finally ready to make use of the `openmc.mgxs` module to generate multi-group cross sections! First, let's define a "fine" 8-group and "coarse" 2-group structures using the built-in `EnergyGroups` class.

In [10]:
# Instantiate a "fine" 8-group EneryGroups object
fine_groups = mgxs.EnergyGroups()
fine_groups.group_edges = np.array([0., 0.058e-6, 0.14e-6, 0.28e-6,
                                    0.625e-6, 4.e-6, 5.53e-3, 821.e-3, 20.])

# Instantiate a "coarse" 2-group EneryGroups object
coarse_groups = mgxs.EnergyGroups()
coarse_groups.group_edges = np.array([0., 0.625e-6, 20.])

We can now use the fine and coarse `EnergyGroups` objects, along with our previously created materials and geometry, to instantiate some `MGXS` objects from the `openmc.mgxs` module. In particular, the following are subclasses of generic and abstract `MGXS` class:

* `TotalXS`
* `TransportXS`
* `AbsorptionXS`
* `CaptureXS`
* `FissionXS`
* `NuFissionXS`
* `ScatterXS`
* `NuScatterXS`
* `ScatterMatrixXS`
* `NuScatterMatrixXS`
* `Chi`

These classes provide us with an interface to generate the tally inputs as well as perform post-processing of OpenMC's tally data to compute the respective multi-group cross sections. In this case, let's create the multi-group cross sections needed to run an OpenMOC simulation to verify the accuracy of our cross sections. In particular, we will define total, nu-fission, nu-scatter and chi cross sections for our infinite medium cell as the domain and our fine 8-group structure as our energy groups.

In [11]:
# Instantiate cross sections needed for an OpenMOC simulation
transport = mgxs.TransportXS(domain=cell, domain_type='cell', groups=fine_groups)
nufission = mgxs.NuFissionXS(domain=cell, domain_type='cell', groups=fine_groups)
nuscatter = mgxs.NuScatterMatrixXS(domain=cell, domain_type='cell', groups=fine_groups)
chi = mgxs.Chi(domain=cell, domain_type='cell', groups=fine_groups)

Next, we must instruct our multi-group cross section objects to generate the tallies needed to calculate each of them in OpenMC. This can be done with the `MGXS.create_tallies()` routine.

In [12]:
# Instruct each multi-group cross section to generate tallies
transport.create_tallies()
nufission.create_tallies()
nuscatter.create_tallies()
chi.create_tallies()

Each multi-group cross section object stores its tallies in a Python dictionary called `tallies`. We can inspect the tallies in the dictionary for our `NuFission` object as follows. 

In [13]:
nufission.tallies

{'flux': Tally
 	ID             =	10003
 	Name           =	
 	Filters        =	
                 		cell	[1]
                 		energy	[  0.00000000e+00   5.80000000e-08   1.40000000e-07   2.80000000e-07
    6.25000000e-07   4.00000000e-06   5.53000000e-03   8.21000000e-01
    2.00000000e+01]
 	Nuclides       =	total 
 	Scores         =	['flux']
 	Estimator      =	tracklength, 'nu-fission': Tally
 	ID             =	10004
 	Name           =	
 	Filters        =	
                 		cell	[1]
                 		energy	[  0.00000000e+00   5.80000000e-08   1.40000000e-07   2.80000000e-07
    6.25000000e-07   4.00000000e-06   5.53000000e-03   8.21000000e-01
    2.00000000e+01]
 	Nuclides       =	total 
 	Scores         =	['nu-fission']
 	Estimator      =	tracklength}

The `NuFission` object includes tracklength tallies for the 'nu-fission' and 'flux' scores in the 8-group structure in cell 1. Now that each multi-group cross section object contains the tallies that it needs, we must add these tallies to a `TalliesFile` object to generate the "tallies.xml" input file for OpenMC.

In [14]:
# Instantiate an empty TalliesFile
tallies_file = openmc.TalliesFile()

# Add transport tallies to the tallies file
for tally in transport.tallies.values():
    tallies_file.add_tally(tally, merge=True)

# Add nu-fission tallies to the tallies file
for tally in nufission.tallies.values():
    tallies_file.add_tally(tally, merge=True)

# Add nu-scatter tallies to the tallies file
for tally in nuscatter.tallies.values():
    tallies_file.add_tally(tally, merge=True)

# Add chi tallies to the tallies file    
for tally in chi.tallies.values():
    tallies_file.add_tally(tally, merge=True)
                
# Export to "tallies.xml"
tallies_file.export_to_xml()

Now we a have a complete set of inputs, so we can go ahead and run our simulation.

In [ ]:
# Run OpenMC!
executor = openmc.Executor()
executor.run_simulation()


       .d88888b.                             888b     d888  .d8888b.
      d88P" "Y88b                            8888b   d8888 d88P  Y88b
      888     888                            88888b.d88888 888    888
      888     888 88888b.   .d88b.  88888b.  888Y88888P888 888       
      888     888 888 "88b d8P  Y8b 888 "88b 888 Y888P 888 888       
      888     888 888  888 88888888 888  888 888  Y8P  888 888    888
      Y88b. .d88P 888 d88P Y8b.     888  888 888   "   888 Y88b  d88P
       "Y88888P"  88888P"   "Y8888  888  888 888       888  "Y8888P"
__________________888______________________________________________________
                  888
                  888

      Copyright:      2011-2015 Massachusetts Institute of Technology
      License:        http://mit-crpg.github.io/openmc/license.html
      Version:        0.7.0
      Git SHA1:       23535afa1c69644bb299bde18a094c3b99d53ae0
      Date/Time:      2015-10-08 19:25:10
      MPI Processes:  1

 =======================

### Tally Data Processing

Our simulation ran successfully and created a statepoint file with all the tally data in it. We begin our analysis here loading the statepoint file and 'reading' the results. By default, data from the statepoint file is only read into memory when it is requested. This helps keep the memory use to a minimum even when a statepoint file may be huge.

In [ ]:
# Load the last statepoint file
sp = openmc.StatePoint('statepoint.50.h5')

In addition to the statepoint file, our simulation also created a summary file which encapsulates information about the materials and geometry which is necessary for the `openmc.mgxs` module to properly process the tally data. We first create a summary object and link it with the statepoint.

In [ ]:
# Load the summary file and link it with the statepoint
su = openmc.Summary('summary.h5')
sp.link_with_summary(su)

The statepoint is now ready to be analyzed by our multi-group cross sections. The first step is to load the tallies from the statepoint into each object.

In [ ]:
# Load the tallies from the statepoint into each MGXS object
transport.load_from_statepoint(sp)
nufission.load_from_statepoint(sp)
nuscatter.load_from_statepoint(sp)
chi.load_from_statepoint(sp)

The multi-group cross section objects can now use OpenMC's [tally arithmetic](http://mit-crpg.github.io/openmc/pythonapi/examples/pandas-dataframes.html) to compute cross sections from the tally data.

In [ ]:
transport.compute_xs()
nufission.compute_xs()
nuscatter.compute_xs()
chi.compute_xs()

Voila! Our multi-group cross sections are now ready to rock 'n roll!

### Cross Section Data Visualization

Let's first inspect our fission production cross section by printing it to the screen.

In [ ]:
nufission.print_xs()

Since the `openmc.mgxs` module uses tally arithmetic under-the-hood, the cross section is stored as a "derived" tally. This means that it can be queried and manipulated using all of the same method supported for the `Tally` class in the OpenMC Python API. For example, we can construct a Pandas DataFrame of the multi-group cross section data.

In [ ]:
df = nuscatter.get_pandas_dataframe()
df.head(10)

Each multi-group cross section object can be easily exported to a variety of file formats, including CSV, Excel, and LaTeX for storage or data processing.

In [ ]:
transport.export_xs_data(filename='transport-xs', format='excel')

The following code snippet shows how to export all of four cross sections to the same HDF5 binary data store.

In [ ]:
transport.build_hdf5_store(filename='mgxs', append=True)
nufission.build_hdf5_store(filename='mgxs', append=True)
nuscatter.build_hdf5_store(filename='mgxs', append=True)
chi.build_hdf5_store(filename='mgxs', append=True)

### Verification with OpenMOC

Of course it is always a good idea to verify that one's cross sections are accurate. We can easily do so here with the deterministic transport code OpenMOC. First, we will use OpenCG to reconstruct our OpenMC geometry from the summary file into a equivalent OpenMOC geometry.

In [ ]:
# Import OpenMOC and the OpenMOC/OpenCG compatibility module
import openmoc
from openmoc.compatible import get_openmoc_geometry

# Create an OpenCG Geometry from the OpenMC Geometry stored in the summary
su.make_opencg_geometry()

# Create an OpenMOC Geometry from the OpenCG Geometry
openmoc_geometry = get_openmoc_geometry(su.opencg_geometry)

Now, we can inject the multi-group cross sections into the equivalent infinite homogeneous medium OpenMOC geometry.

In [ ]:
# Get all OpenMOC cells in the gometry
openmoc_cells = openmoc_geometry.getRootUniverse().getAllCells()

# Inject multi-group cross sections into OpenMOC Materials
for cell_id, cell in openmoc_cells.items():
    
    # Get a reference to the Material filling this Cell
    openmoc_material = cell.getFillMaterial()
    
    # Set the number of energy groups for the Material
    openmoc_material.setNumEnergyGroups(fine_groups.num_groups)
    
    # Inject NumPy arrays of cross section data into the Material
    openmoc_material.setSigmaT(transport.get_xs().flatten())
    openmoc_material.setNuSigmaF(nufission.get_xs().flatten())
    openmoc_material.setSigmaS(nuscatter.get_xs().flatten())
    openmoc_material.setChi(chi.get_xs().flatten())

We are now ready to run OpenMOC to verify our cross-sections from OpenMC.

In [ ]:
# Generate tracks for OpenMOC
openmoc_geometry.initializeFlatSourceRegions()
track_generator = openmoc.TrackGenerator(openmoc_geometry, 128, 0.1)
track_generator.generateTracks()

# Run OpenMOC
solver = openmoc.CPUSolver(track_generator)
solver.computeEigenvalue()

We report the eigenvalues computed by OpenMC and OpenMOC here together to summarize our results.

In [ ]:
# Print report of keff and bias with OpenMC
openmoc_keff = solver.getKeff()
openmc_keff = sp.k_combined[0]
bias = (openmoc_keff - openmc_keff) * 1e5

print('openmc keff = {0:1.6f}'.format(openmc_keff))
print('openmoc keff = {0:1.6f}'.format(openmoc_keff))
print('bias [pcm]: {0:1.1f}'.format(bias))

Although there is a non-trivial bias, one can easily run the preceding code with more particle histories to show that both codes converge to the same eigenvalue with <10 pcm bias. It should be noted that this discrepancy is due to use of tracklength tallies for `NuFission`, while one must use more slowly converging analog tallies for `TransportXS`, `NuScatterMatrixXS` and `Chi` (which require an 'energyout' filter).

# Fuel Pin Cell

In this section we show how to compute multi-group cross sections for a fuel pin cell. In addition, we will illustrate how to use some of the more advanced features in `openmc.mgxs` such as nuclide-by-nuclide microscopic cross section tallies and downstream energy group condensation.

### Generate Inputs

this time we separate our nuclides into three distinct materials for water, clad and fuel.

In [ ]:
# 1.6 enriched fuel
fuel = openmc.Material(name='1.6% Fuel')
fuel.set_density('g/cm3', 10.31341)
fuel.add_nuclide(u235, 3.7503e-4)
fuel.add_nuclide(u238, 2.2625e-2)
fuel.add_nuclide(o16, 4.6007e-2)

# borated water
water = openmc.Material(name='Borated Water')
water.set_density('g/cm3', 0.740582)
water.add_nuclide(h1, 4.9457e-2)
water.add_nuclide(o16, 2.4732e-2)

# zircaloy
zircaloy = openmc.Material(name='Zircaloy')
zircaloy.set_density('g/cm3', 6.55)
zircaloy.add_nuclide(zr90, 7.2758e-3)

With our materials, we can now create a materials file object that can be exported to an actual XML file.

In [ ]:
# Instantiate a MaterialsFile, add Materials
materials_file = openmc.MaterialsFile()
materials_file.add_material(fuel)
materials_file.add_material(water)
materials_file.add_material(zircaloy)
materials_file.default_xs = '71c'

# Export to "materials.xml"
materials_file.export_to_xml()

Now let's move on to the geometry. Our problem will have three regions for the fuel, the clad, and the surrounding coolant. The first step is to create the bounding surfaces -- in this case two cylinders and six reflective planes.

In [ ]:
# Create cylinders for the fuel and clad
fuel_outer_radius = openmc.ZCylinder(x0=0.0, y0=0.0, R=0.39218)
clad_outer_radius = openmc.ZCylinder(x0=0.0, y0=0.0, R=0.45720)

# Create boundary planes to surround the geometry
# Use both reflective and vacuum boundaries to make life interesting
min_x = openmc.XPlane(x0=-0.63, boundary_type='reflective')
max_x = openmc.XPlane(x0=+0.63, boundary_type='reflective')
min_y = openmc.YPlane(y0=-0.63, boundary_type='reflective')
max_y = openmc.YPlane(y0=+0.63, boundary_type='reflective')
min_z = openmc.ZPlane(z0=-0.63, boundary_type='reflective')
max_z = openmc.ZPlane(z0=+0.63, boundary_type='reflective')

With the surfaces defined, we can now create cells that are defined by intersections of half-spaces created by the surfaces.

In [ ]:
# Create a Universe to encapsulate a fuel pin
pin_cell_universe = openmc.Universe(name='1.6% Fuel Pin')

# Create fuel Cell
fuel_cell = openmc.Cell(name='1.6% Fuel')
fuel_cell.fill = fuel
fuel_cell.add_surface(fuel_outer_radius, halfspace=-1)
pin_cell_universe.add_cell(fuel_cell)

# Create a clad Cell
clad_cell = openmc.Cell(name='1.6% Clad')
clad_cell.fill = zircaloy
clad_cell.add_surface(fuel_outer_radius, halfspace=+1)
clad_cell.add_surface(clad_outer_radius, halfspace=-1)
pin_cell_universe.add_cell(clad_cell)

# Create a moderator Cell
moderator_cell = openmc.Cell(name='1.6% Moderator')
moderator_cell.fill = water
moderator_cell.add_surface(clad_outer_radius, halfspace=+1)
pin_cell_universe.add_cell(moderator_cell)

OpenMC requires that there is a "root" universe. Let us create a root cell that is filled by the pin cell universe and then assign it to the root universe.

In [ ]:
# Create root Cell
root_cell = openmc.Cell(name='root cell')
root_cell.fill = pin_cell_universe

# Add boundary planes
root_cell.add_surface(min_x, halfspace=+1)
root_cell.add_surface(max_x, halfspace=-1)
root_cell.add_surface(min_y, halfspace=+1)
root_cell.add_surface(max_y, halfspace=-1)

# Create root Universe
root_universe = openmc.Universe(universe_id=0, name='root universe')
root_universe.add_cell(root_cell)

We now must create a geometry that is assigned a root universe, put the geometry into a geometry file, and export it to XML.

In [ ]:
# Create Geometry and set root Universe
openmc_geometry = openmc.Geometry()
openmc_geometry.root_universe = root_universe

# Instantiate a GeometryFile
geometry_file = openmc.GeometryFile()
geometry_file.geometry = openmc_geometry

# Export to "geometry.xml"
geometry_file.export_to_xml()

We will reuse our settings from the previous simulation. Now, we let's create transport, nu-fission, nu-scatter and chi multi-group cross sections for each cell.

In [ ]:
# Extract all Cells filled by Materials
openmc_cells = openmc_geometry.get_all_material_cells()

# Create dictionary to store multi-group cross sections for all cells
xs_library = {}

# Instantiate 8-group cross sections for each cell
for cell in openmc_cells:
    xs_library[cell.id] = {}
    xs_library[cell.id]['transport']  = mgxs.TransportXS(groups=fine_groups)
    xs_library[cell.id]['nu-fission'] = mgxs.NuFissionXS(groups=fine_groups)
    xs_library[cell.id]['nu-scatter'] = mgxs.NuScatterMatrixXS(groups=fine_groups)
    xs_library[cell.id]['chi'] = mgxs.Chi(groups=fine_groups)

In this case, we did not give our cross sections a spatial domain in their constructors. Instead, we will loop over all cells to set each cross sections domain. In addition, we will set each cross section to tally cross sections on a per-nuclide basis through the use of the `by_nuclide` instance attribute.  

In [ ]:
# Instantiate an empty TalliesFile
tallies_file = openmc.TalliesFile()

# Iterate over all cells and cross section types
for cell in openmc_cells:
    for rxn_type in xs_library[cell.id].keys():

        # Set the cross sections domain type to the cell
        xs_library[cell.id][rxn_type].domain = cell
        xs_library[cell.id][rxn_type].domain_type = 'cell'
        
        # Tally cross sections by nuclide (e.g., micro cross sections)
        xs_library[cell.id][rxn_type].by_nuclide = True
        
        # Create OpenMC tallies for this cross section
        xs_library[cell.id][rxn_type].create_tallies()
                
        # Add OpenMC tallies to the tallies file for XML generation
        for tally in xs_library[cell.id][rxn_type].tallies.values():
            tallies_file.add_tally(tally, merge=True)

# Export to "tallies.xml"
tallies_file.export_to_xml()

Now we a have a complete set of inputs, so we can go ahead and run our simulation.

In [ ]:
# Delete old HDF5 files
!rm *.h5

# Run OpenMC with the output throttled!
executor = openmc.Executor()
executor.run_simulation(output=False)

### Tally Data Processing

Our simulation ran successfully and created a statepoint file with all the tally data in it. As before, we begin our analysis here loading the statepoint file.

In [ ]:
# Load the last statepoint and summary files
sp = openmc.StatePoint('statepoint.50.h5')
su = openmc.Summary('summary.h5')
sp.link_with_summary(su)

The statepoint is now ready to be analyzed by our multi-group cross sections. Next, we load the tallies from the statepoint into each object and to compute the cross sections using tally arithmetic.

In [ ]:
# Iterate over all cells and cross section types
for cell in openmc_cells:
    for rxn_type in xs_library[cell.id].keys():
        xs_library[cell.id][rxn_type].load_from_statepoint(sp)
        xs_library[cell.id][rxn_type].compute_xs()

That's it! Our multi-group cross sections are now ready for the big spotlight. This time we have cross sections in three distinct spatial zones - fuel, clad and moderator - on a per-nuclide basis.

### Cross Section Data Visualization

Let's first inspect one of our cross sections by printing it to the screen as a microscopic cross section in units of barns.

In [ ]:
nufission = xs_library[fuel_cell.id]['nu-fission']
nufission.print_xs(xs_type='micro', nuclides=['U-235', 'U-238'])

Our multi-group cross sections are capable of summing across all nuclides to provide us with macroscopic cross sections as well.

In [ ]:
nufission = xs_library[fuel_cell.id]['nu-fission']
nufission.print_xs(xs_type='macro', nuclides='sum')

Although a printed report is nice, it is not scalable or flexible. Let's extract the cross section data for the moderator as a Pandas DataFrame.

In [ ]:
nuscatter = xs_library[moderator_cell.id]['nu-scatter']
df = nuscatter.get_pandas_dataframe(xs_type='micro')
df.head(10)

We can easily use the Pandas DataFrame to extract the H-1 and O-16 scattering matrices separately.

In [ ]:
# Slice DataFrame in two for each nuclide's mean values
h1 = df[df['nuclide'] == 'H-1']['mean']
o16 = df[df['nuclide'] == 'O-16']['mean']

# Cast DataFrames as NumPy arrays
h1 = h1.as_matrix()
o16 = o16.as_matrix()

# Reshape arrays to 2D matrix for plotting
h1.shape = (fine_groups.num_groups, fine_groups.num_groups)
o16.shape = (fine_groups.num_groups, fine_groups.num_groups)

Matplotlib's `imshow` routine can be used to plot the matrices to illustrate their sparsity structures.

In [ ]:
# Create plot of the H-1 scattering matrix
fig = plt.subplot(121)
fig.imshow(h1, interpolation='nearest')
plt.title('H-1 Scattering Matrix')

# Create plot of the O-16 scattering matrix
fig2 = plt.subplot(122)
fig2.imshow(o16, interpolation='nearest')
plt.title('O-16 Scattering Matrix')

# Show the plot on screen
plt.show()

Next, we illustate how one can easily take multi-group cross sections and condense them down to a coarser energy group structure using. The `get_condensed_xs(...)` class method takes in as a parameter an `EnergyGroups` object with a coarse(r) group structure and returns a new multi-group cross section condensed to the coarse groups. We illustrate this process below using the 2-group structure created earlier.

In [ ]:
# Extract the 16-group transport cross section for the fuel
fine_xs = xs_library[fuel_cell.id]['transport']

# Condense to the 2-group structure
condense_xs = fine_xs.get_condensed_xs(coarse_groups)

Group condensation is as simple as that! We now have a new coarse 2-group cross section in addition to our original 16-group cross section. Let's inspect the 2-group cross section by printing it to the screen and extracting a Pandas DataFrame as we have already learned how to do.

In [ ]:
condense_xs.print_xs()

In [ ]:
df = condense_xs.get_pandas_dataframe(xs_type='micro')
df

### Verification with OpenMOC

Finally, let's verify our cross sections using OpenMOC. First, we use OpenCG construct an equivalent OpenMOC geometry just as we did before.

In [ ]:
# Create an OpenCG Geometry from the OpenMC Geometry stored in the summary
su.make_opencg_geometry()

# Create an OpenMOC Geometry from the OpenCG Geometry
openmoc_geometry = get_openmoc_geometry(su.opencg_geometry)

Likewise, we can inject the multi-group cross sections into the equivalent fuel pin cell OpenMOC geometry.

In [ ]:
# Get all OpenMOC cells in the gometry
openmoc_cells = openmoc_geometry.getRootUniverse().getAllCells()

# Inject multi-group cross sections into OpenMOC Materials
# NOTE: This code will work for 1, 10, or 1,000s of cells
# as is the case for a complicated geometry like BEAVRS
for cell_id, cell in openmoc_cells.items():
    
    # Ignore the root cell
    if cell.getName() == 'root cell':
        continue
    
    # Get a reference to the Material filling this Cell
    openmoc_material = cell.getFillMaterial()
    
    # Set the number of energy groups for the Material
    openmoc_material.setNumEnergyGroups(fine_groups.num_groups)
    
    # Extract the appropriate cross section objects for this cell
    transport = xs_library[cell_id]['transport']
    nufission = xs_library[cell_id]['nu-fission']
    nuscatter = xs_library[cell_id]['nu-scatter']
    chi = xs_library[cell_id]['chi']
    
    # Inject NumPy arrays of cross section data into the Material
    # NOTE: In each case we must sum across nuclides to get the
    # macroscopic cross sections needed by OpenMOC
    openmoc_material.setSigmaT(transport.get_xs(nuclides='sum').flatten())
    openmoc_material.setNuSigmaF(nufission.get_xs(nuclides='sum').flatten())
    openmoc_material.setSigmaS(nuscatter.get_xs(nuclides='sum').flatten())
    openmoc_material.setChi(chi.get_xs(nuclides='sum').flatten())

We are now ready to run OpenMOC to verify our cross-sections from OpenMC.

In [ ]:
# Throttle OpenMOC output to screen
openmoc.log.set_log_level('WARNING')

# Generate tracks for OpenMOC
openmoc_geometry.initializeFlatSourceRegions()
track_generator = openmoc.TrackGenerator(openmoc_geometry, 128, 0.1)
track_generator.generateTracks()

# Run OpenMOC
solver = openmoc.CPUSolver(track_generator)
solver.computeEigenvalue()

We report the eigenvalues computed by OpenMC and OpenMOC here together to summarize our results.

In [ ]:
# Print report of keff and bias with OpenMC
openmoc_keff = solver.getKeff()
openmc_keff = sp.k_combined[0]
bias = (openmoc_keff - openmc_keff) * 1e5

print('openmc keff = {0:1.6f}'.format(openmc_keff))
print('openmoc keff = {0:1.6f}'.format(openmoc_keff))
print('bias [pcm]: {0:1.1f}'.format(bias))

As a sanity check, let's run a simulation with the coarse 2-group cross sections to ensure that they produce a reasonable result.

In [ ]:
su.make_opencg_geometry()
openmoc_geometry = get_openmoc_geometry(su.opencg_geometry)
openmoc_cells = openmoc_geometry.getRootUniverse().getAllCells()

# Inject multi-group cross sections into OpenMOC Materials
for cell_id, cell in openmoc_cells.items():
    
    # Ignore the root cell
    if cell.getName() == 'root cell':
        continue
    
    openmoc_material = cell.getFillMaterial()
    openmoc_material.setNumEnergyGroups(coarse_groups.num_groups)
    
    # Extract the appropriate cross section objects for this cell
    transport = xs_library[cell_id]['transport']
    nufission = xs_library[cell_id]['nu-fission']
    nuscatter = xs_library[cell_id]['nu-scatter']
    chi = xs_library[cell_id]['chi']
    
    # Perform group condensation
    transport = transport.get_condensed_xs(coarse_groups)
    nufission = nufission.get_condensed_xs(coarse_groups)
    nuscatter = nuscatter.get_condensed_xs(coarse_groups)
    chi = chi.get_condensed_xs(coarse_groups)
    
    # Inject NumPy arrays of cross section data into the Material
    openmoc_material.setSigmaT(transport.get_xs(nuclides='sum').flatten())
    openmoc_material.setNuSigmaF(nufission.get_xs(nuclides='sum').flatten())
    openmoc_material.setSigmaS(nuscatter.get_xs(nuclides='sum').flatten())
    openmoc_material.setChi(chi.get_xs(nuclides='sum').flatten())

In [ ]:
# Generate tracks for OpenMOC
openmoc_geometry.initializeFlatSourceRegions()
track_generator = openmoc.TrackGenerator(openmoc_geometry, 128, 0.1)
track_generator.generateTracks()

# Run OpenMOC
solver = openmoc.CPUSolver(track_generator)
solver.computeEigenvalue()

In [ ]:
# Print report of keff and bias with OpenMC
openmoc_keff = solver.getKeff()
openmc_keff = sp.k_combined[0]
bias = (openmoc_keff - openmc_keff) * 1e5

print('openmc keff = {0:1.6f}'.format(openmc_keff))
print('openmoc keff = {0:1.6f}'.format(openmoc_keff))
print('bias [pcm]: {0:1.1f}'.format(bias))

There is a non-trivial bias in both the 2-group and 8-group cases. In the case of the pin cell, one can show that these biases do not converge to <100 pcm with more particle histories. In the case of heterogeneous geometries, additional measures must be taken to address the following three sources of bias:

* Appropriate transport-corrected cross sections
* Spatial discretization of OpenMOC's mesh
* Constant-in-angle multi-group cross sections